In [1]:
import os
import pandas as pd

In [2]:
from google.cloud import bigquery

project = 'bcs-csw-core'
client = bigquery.Client(project=project)

# Perform a query.
QUERY = """
-- set constants
WITH declares AS (
  SELECT 
    DATE '2024-01-01' AS start_date,
    2024 AS target_planting_year,
    '10002' AS station, -- station where serial number is separated into hopper and station
    ARRAY<STRING> ['Corn', 'Cotton', 'Wheat', 'Soybean', 'Soybeans', 'Sweet Corn'] AS crops,
    ARRAY<STRING> ['10002-004', '10002-041', '10002-048', '10002-049', '10002-053', '10002-072', '10002-075', '10002-076', '10002-079', '10002-085'] AS WIB_machines, -- WIB machine
    ARRAY<STRING> ['10002-081', '10002-001', '10002-005', '10002-028', '10002-019', '10002-020', '10002-071', '10002-084',
                  '10002-029', '10002-058', '10002-062', '10002-030', '10002-059', '10002-057', '10002-063',
                  '10002-061', '10002-056', '10002-065', '10002-082', '10002-002', '10002-023', '10002-022', '10002-024',
                  '10002-064', '10002-027', '10002-055', '10002-026', '10002-088', '10002-025', '10002-090'] AS HAZ_machines,

ARRAY<STRING> ['10002-019','10002-020','10002-022','10002-025','10002-029','10002-055','10002-062','10002-063','10002-065','10002-071','10002-081','10002-084','10002-088','10002-090'] AS CAT3_machines,
ARRAY<STRING> ['10002-001','10002-002','10002-005','10002-023','10002-024','10002-026','10002-027','10002-028','10002-030','10002-056','10002-057','10002-058','10002-059','10002-061','10002-064','10002-082'] AS CAT2_machines,
),

-- cast machine ids to ints, add site label
machine_decoder AS (
  
  SELECT *,
  LTRIM(REGEXP_EXTRACT(machines, r'^[a-zA-Z0-9_.+-]+\-([a-zA-Z0-9-.]+$)'), '0') AS hopper,
  CASE WHEN (machines IN (SELECT WIB_machines FROM declares, UNNEST(WIB_machines) AS WIB_machines))
    THEN 'WIB' ELSE 'HAZ' END AS site,
    CASE WHEN (machines IN (SELECT CAT3_machines FROM declares, UNNEST(CAT3_machines) AS CAT3_machines))
    THEN 'Cat 3' 
     WHEN (machines IN (SELECT CAT2_machines FROM declares, UNNEST(CAT2_machines) AS CAT2_machines))
    THEN 'Cat 2'
    END AS category
  FROM (
    SELECT machines  
    FROM declares,
    UNNEST(ARRAY_CONCAT(declares.HAZ_machines, declares.WIB_machines)) AS machines
  )

),

-- all dates
-- changed this trying to clean up the output, but it didn't work like I expected
date_range AS (
  SELECT * 
  FROM (SELECT DATE AS day
        FROM UNNEST(GENERATE_DATE_ARRAY(DATE '2024-01-01', DATE '2024-12-30', INTERVAL 1 DAY)) AS date
        )
  CROSS JOIN (
      SELECT site
      FROM UNNEST(ARRAY<STRING>['WIB', 'HAZ']) AS site  -- move to declares
  )
),


HAZ_filled_cells AS (
SELECT
         planting_session_id,
         site,
         category,
         cassette_id, 
         cell_number,
         inventory_bid
  FROM `bcs-breeding-datasets.breeding_operations.influx_field_cassette_as_filled_cell` flex
 
  INNER JOIN (SELECT site, hopper, category FROM machine_decoder) AS decoder
    ON flex.hopper = decoder.hopper

  WHERE EXTRACT(DATE FROM filled_on) >= (SELECT start_date FROM declares)
  AND flex.station = (SELECT station FROM declares)
),

first_join AS (
SELECT  inv_bid,barcode, MAX(EXTRACT(Date from datetime_start))AS inventory_available_date
FROM `bcs-breeding-datasets.breeding_operations.mactracker_shellmatic_3`  s3
left join `bcs-csw-core.velocity.inventory` inv
on s3.inv_bid = inv.legacyBarcode
where EXTRACT(DATE FROM datetime_start) >= (SELECT start_date FROM declares)
GROUP BY 1,2
),


HAZ_filled_join_created AS (
  SELECT HAZ_filled_cells.*,inventory_available_date, created_on
  FROM HAZ_filled_cells
  LEFT JOIN `bcs-breeding-datasets.breeding_operations.influx_field_cassette_planting_session` ps
  ON ps.id = HAZ_filled_cells.planting_session_id
  LEFT JOIN first_join AS first_join_inv 
  ON first_join_inv.barcode = cast(HAZ_filled_cells.inventory_bid as int64)
),

inv as (SELECT barcode, legacyBarcode
FROM `bcs-breeding-datasets.velocity.inventory`),
find_sender as (
select inv.*, matexc.receiver_inv_barcode, matexc.sender_inv_barcode
from inv 
left join `bcs-csw-core.exadata.midas_material_exchange` matexc
on matexc.receiver_inv_barcode = inv.legacyBarcode),

combine_2_fts_barcode as (
select barcode, 
case when sender_inv_barcode is not null then sender_inv_barcode
else legacyBarcode 
end as fts_code
from find_sender),

second_join as (
select inv_bid, barcode, max(extract(date from datetime_start)) as inventory_available_date_2
from `bcs-breeding-datasets.breeding_operations.mactracker_shellmatic_3` s3
join combine_2_fts_barcode 
on combine_2_fts_barcode.fts_code = s3.inv_bid
where EXTRACT(DATE FROM datetime_start) >= (SELECT start_date FROM declares)
group by 1,2),

HAZ_filled_join_created_2 AS (
  SELECT *,
  CASE WHEN inventory_available_date is not null then inventory_available_date
  else inventory_available_date_2
  end as inventory_available_date_3
  FROM HAZ_filled_join_created
  left JOIN second_join 
  on second_join.barcode = cast(HAZ_filled_join_created.inventory_bid as int64)
),

RTE_date_table AS (
  SELECT setId,MAX(EXTRACT(Date from audit_time_stamp)) AS RTE_date
  FROM `bcs-breeding-datasets.velocity.set_audits`
  where audit_status = 'Material Fulfillment: Ready to Execute'
  GROUP BY setId
),

HAZ_filled_created_plotrow AS (
  SELECT HAZ_filled_join_created_2.*, type, plrow.set_id,plrow.set_name,RTE_date
  FROM HAZ_filled_join_created_2
  JOIN `bcs-breeding-datasets.breeding_operations.influx_field_cassette_cell` cell 
  ON HAZ_filled_join_created_2.cassette_id = cell.cassette_id and HAZ_filled_join_created_2.cell_number = cell.cell_number
  JOIN `bcs-breeding-datasets.breeding_operations.influx_field_cassette_plot_row` plrow
  ON cell.plot_row_id = plrow.id 
  LEFT JOIN RTE_date_table 
  ON RTE_date_table.setId = plrow.set_id)

 , output as (
  select distinct cassette_id,cell_number,set_id,set_name,
  case when type in ('border','buffer') then type
  else category 
  end as cat_type,
  planting_session_id,
  inventory_available_date_3,
  RTE_date,
  EXTRACT(Date from created_on) AS cassette_finalized_date,
    
  from HAZ_filled_created_plotrow
  where site = 'HAZ'
  order by cassette_id, cell_number   )

  select *
  from output
order by 1,2



"""
output = client.query(QUERY).result().to_dataframe() 



/usr/local/lib/python3.11/dist-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [3]:
output.tail()

,cassette_id,cell_number,set_id,set_name,cat_type,planting_session_id,inventory_available_date_3,RTE_date,cassette_finalized_date
894229,ffff0c68-9da0-4f4a-8118-55ccd483c624,36,<NA>,None,buffer,503c79c4-3875-42a7-ac94-148c4283c74e,NaT,NaT,2024-03-28
894230,ffff0c68-9da0-4f4a-8118-55ccd483c624,37,<NA>,None,buffer,503c79c4-3875-42a7-ac94-148c4283c74e,NaT,NaT,2024-03-28
894231,ffff0c68-9da0-4f4a-8118-55ccd483c624,38,<NA>,None,buffer,503c79c4-3875-42a7-ac94-148c4283c74e,NaT,NaT,2024-03-28
894232,ffff0c68-9da0-4f4a-8118-55ccd483c624,39,<NA>,None,buffer,503c79c4-3875-42a7-ac94-148c4283c74e,NaT,NaT,2024-03-28
894233,ffff0c68-9da0-4f4a-8118-55ccd483c624,40,<NA>,None,buffer,503c79c4-3875-42a7-ac94-148c4283c74e,NaT,NaT,2024-03-28


In [ ]:
# output.to_excel('3 Dates by Category.xlsx')

In [4]:
output.isnull().sum()

cassette_id                        0
cell_number                        0
set_id                        289555
set_name                      289555
cat_type                           0
planting_session_id                0
inventory_available_date_3    551738
RTE_date                      289555
cassette_finalized_date            0
dtype: int64

In [22]:
output.shape

(894234, 9)

In [61]:
inv_avl = output.groupby(['inventory_available_date_3','cat_type']).size().reset_index(name='n_cells')
inv_avl_pivot = inv_avl.pivot(index="inventory_available_date_3", columns="cat_type", values="n_cells")

In [62]:
rte = output.groupby(['RTE_date','cat_type']).size().reset_index(name='n_cells')
rte_pivot = rte.pivot(index="RTE_date", columns="cat_type", values="n_cells")

In [63]:
c_finalized = output.groupby(['cassette_finalized_date','cat_type']).size().reset_index(name='n_cells')
c_finalized_pivot = c_finalized.pivot(index="cassette_finalized_date", columns="cat_type", values="n_cells")

In [5]:
# subset to only 2024 
gl = pd.read_csv('HSC Greenlight System.csv')
gl['Actionable Date'] = pd.to_datetime(gl['Actionable Date'])
gl['modified_date'] = pd.to_datetime(gl.Modified.str.split(' ').str[0])
gl['Greenlight Date'] =gl['Actionable Date'].fillna(gl['modified_date'])
gl_2024 = gl[(gl['Greenlight Date'].dt.year == 2024) & (
    gl['Planning Status:'] == 'Ready To Package') & (gl['Material Disposition'].str.contains('Velocity'))].copy()

In [9]:
# Assuming 'DirtyColumn' is the column with messy data
dirty_column = gl_2024['Set IDs/Set Names']

In [10]:
import re

def extract_data_with_range(row, index):
    ids = []
    block_names = []
    set_names = []
    other_columns = {}
    
    # Regex patterns
    id_pattern = r'\b\d+\b'  # Matches sequences of digits
    block_pattern = r'\b\w+_\w+\b'  # Matches words separated by underscore
    name_pattern = r'\b\w+ L\d+\b'  # Matches words followed by " Lxx"
    range_pattern = r'(\d+)\s*-\s*(\d+)\s*\(.*?(\d+)\s*sets.*?\)'  # Matches ranges like "480664 - 480669 (6 sets)"
    
    # Find all Sets Set Ids
    ids.extend(re.findall(id_pattern, row))
    
    # Find all Experiment Block Names
    block_names.extend(re.findall(block_pattern, row))
    
    # Find all Set Names
    set_names.extend(re.findall(name_pattern, row))
    
    # Handle range cases
    ranges = re.findall(range_pattern, row)
    for start, end, count in ranges:
        start = int(start)
        end = int(end)
        count = int(count)
        if end >= start and count > 1:
            ids.extend(map(str, range(start, end + 1)))
    
    # Extract other columns from original DataFrame
    other_columns = {col: gl_2024.loc[index, col] for col in gl_2024.columns if col != 'DirtyColumn'}
    
    return ids, block_names, set_names, other_columns


In [11]:
cleaned_data = []

for index, row in dirty_column.items():
    ids, block_names, set_names, other_columns = extract_data_with_range(row, index)
    
    # Append new rows based on extracted data
    for id in ids:
        new_row = {'Set Id': id, 'Experiment Block Name': None, 'Set Name': None}
        new_row.update(other_columns)  # Add other columns
        cleaned_data.append(new_row)
    
    for block_name in block_names:
        new_row = {'Set Id': None, 'Experiment Block Name': block_name, 'Set Name': None}
        new_row.update(other_columns)  # Add other columns
        cleaned_data.append(new_row)
    
    for set_name in set_names:
        new_row = {'Set Id': None, 'Experiment Block Name': None, 'Set Name': set_name}
        new_row.update(other_columns)  # Add other columns
        cleaned_data.append(new_row)

# Create a new DataFrame with cleaned data
cleaned_gl_2024 = pd.DataFrame(cleaned_data)

In [12]:
cleaned_gl_2024['Set Id'] = pd.to_numeric(cleaned_gl_2024['Set Id'])

In [14]:
cleaned_gl_2024 = cleaned_gl_2024.drop_duplicates()

In [15]:
cleaned_gl_2024.shape

(6429, 31)

In [16]:
final_gl_2024 = cleaned_gl_2024[['Set Id','Experiment Block Name', 'Set Name','Greenlight Date']]

In [17]:
final_gl_2024.shape

(6429, 4)

In [24]:
# Filter to include only non-null rows in the join key
output_filtered = output[output['set_id'].notnull()]

# Perform left join
merged_df1 = pd.merge(output_filtered, final_gl_2024, left_on='set_id', right_on='Set Id', how='left')

In [33]:
merged_df1.shape

(604679, 13)

In [34]:
merged_df1.tail(2)

,cassette_id,cell_number,set_id,set_name,cat_type,planting_session_id,inventory_available_date_3,RTE_date,cassette_finalized_date,Set Id,Experiment Block Name,Set Name,Greenlight Date
604677,fff0b09a-5b5e-448d-8f6b-1c23ec6f30d4,119,419451,RCN_REITDR L01,Cat 3,0c977944-e328-4bd5-8378-153548126603,NaT,2024-03-26,2024-03-29,419451.0,None,None,2024-03-25
604678,fff0b09a-5b5e-448d-8f6b-1c23ec6f30d4,120,419451,RCN_REITDR L01,border,0c977944-e328-4bd5-8378-153548126603,NaT,2024-03-26,2024-03-29,419451.0,None,None,2024-03-25


In [31]:
# Checked and found that rows having set_id would have set_name
# so we don't need to do the merge for the rest

output[(output['set_id'].isnull()) & (output['set_name'].notnull())]

,cassette_id,cell_number,set_id,set_name,cat_type,planting_session_id,inventory_available_date_3,RTE_date,cassette_finalized_date


In [32]:
rest_output = output[output['set_id'].isnull()]
rest_output.shape

(289555, 9)

In [45]:
rest_output.head(2)

,cassette_id,cell_number,set_id,set_name,cat_type,planting_session_id,inventory_available_date_3,RTE_date,cassette_finalized_date
0,00100684-1245-42c2-94dc-ebbdf5b071f4,1,<NA>,None,buffer,63457052-f02d-475f-b39b-429943d99128,NaT,NaT,2024-03-29
1,00100684-1245-42c2-94dc-ebbdf5b071f4,2,<NA>,None,buffer,63457052-f02d-475f-b39b-429943d99128,NaT,NaT,2024-03-29


In [35]:
# In the merged_df1, see if there's any rows that have gl_date null, for them, try merge by set name
filter2 = merged_df1[merged_df1['Greenlight Date'].isnull()]
filter2.shape

(5712, 13)

In [39]:
filter2.columns

Index(['cassette_id', 'cell_number', 'set_id', 'set_name', 'cat_type',
       'planting_session_id', 'inventory_available_date_3', 'RTE_date',
       'cassette_finalized_date', 'Set Id', 'Experiment Block Name',
       'Set Name', 'Greenlight Date'],
      dtype='object')

In [40]:
merged_df2 = pd.merge(filter2[['cassette_id', 'cell_number', 'set_id', 'set_name', 'cat_type',
       'planting_session_id', 'inventory_available_date_3', 'RTE_date',
       'cassette_finalized_date']], final_gl_2024, left_on='set_name', right_on='Set Name', how='left')
merged_df2.shape

(5712, 13)

In [42]:
merged_df2.head(2)

,cassette_id,cell_number,set_id,set_name,cat_type,planting_session_id,inventory_available_date_3,RTE_date,cassette_finalized_date,Set Id,Experiment Block Name,Set Name,Greenlight Date
0,01d021e4-1574-4674-8a29-16f2f7750223,3,488137,FTC_NCF105 L06,Cat 3,f2f0cce1-f364-421a-9004-d41b322c9307,2024-01-19,2024-03-14,2024-03-15,NaN,None,FTC_NCF105 L06,2024-03-19
1,01d021e4-1574-4674-8a29-16f2f7750223,4,488137,FTC_NCF105 L06,Cat 3,f2f0cce1-f364-421a-9004-d41b322c9307,NaT,2024-03-14,2024-03-15,NaN,None,FTC_NCF105 L06,2024-03-19


In [46]:
filter3 = merged_df2[merged_df2['Greenlight Date'].isnull()]
filter3.shape

(0, 13)

In [50]:
# Concatenate all merged dataframes
final_df = pd.concat([merged_df1, merged_df2, rest_output], ignore_index=True)

In [53]:
duplicate_rows = final_df[final_df.duplicated(subset=['cassette_id', 'cell_number'], keep=False)]
sorted_duplicate_rows = duplicate_rows.sort_values(by=['cassette_id', 'cell_number'])
rows_to_delete = sorted_duplicate_rows[sorted_duplicate_rows['Greenlight Date'].isnull()]

In [54]:
rows_to_delete.shape

(5712, 13)

In [58]:
mask = final_df.isin(rows_to_delete.to_dict('list')).all(axis=1)

# Remove rows from df1 that are in df2
final_output = final_df[~mask]

In [59]:
final_output.shape

(894234, 13)

In [60]:
gl_date = final_output.groupby(['Greenlight Date','cat_type']).size().reset_index(name='n_cells')
gl_date_pivot = gl_date.pivot(index="Greenlight Date", columns="cat_type", values="n_cells")

In [64]:
with pd.ExcelWriter('HAZ Daily Capacity by Category.xlsx') as writer:
    final_output.to_excel(writer, sheet_name='Cassette-Cell-Category')
    inv_avl_pivot.to_excel(writer, sheet_name='Inventory Available Date')
    rte_pivot.to_excel(writer, sheet_name='RTE Date')
    c_finalized_pivot.to_excel(writer, sheet_name='Cassette Finalized Date')
    gl_date_pivot.to_excel(writer, sheet_name='Greenlight Date')